In [25]:
import folium
from folium.plugins import MarkerCluster
import os
import json
import branca.colormap as cm

In [26]:
# gather all geojson files
# geojson files are located in their respective folders under data
# files are then saved in a dictionary with the folder name as key

files = {}

for root, dirs, filenames in os.walk('data'):
    for f in filenames:

        # get the folder name after data\
        # i.e. emobility_json or hafengebiets_json
        folder = root.split('\\')[1]

        # create a new list if the folder name is not in the dictionary
        if folder not in files:
            files[folder] = []
        
        # append the file name to the list
        files[folder].append(f)

In [27]:
files

{'bedrohte_gruppen_json': ['app_perspektive_wohnen_bestehend_EPSG_25832.json',
  'app_perspektive_wohnen_bestehend_EPSG_4326.json',
  'de_hh_up_nicht_staatliche_schulen_EPSG_25832.json',
  'de_hh_up_nicht_staatliche_schulen_EPSG_4326.json',
  'de_hh_up_staatliche_schulen_EPSG_25832.json',
  'de_hh_up_staatliche_schulen_EPSG_4326.json',
  'de_hh_up_vollstationaere_pflegeeinrichtungen_EPSG_25832.json',
  'de_hh_up_vollstationaere_pflegeeinrichtungen_EPSG_4326.json',
  'settings.json'],
 'deichgrundgrenze_json': ['de_hh_up_deichgrundgrenze_EPSG_25832.json',
  'de_hh_up_deichgrundgrenze_EPSG_4326.json',
  'settings.json'],
 'emobility_json': ['app_stromnetz_emobility_EPSG_25832.json',
  'app_stromnetz_emobility_EPSG_4326.json',
  'settings.json'],
 'feuerwehrstandorte_json': ['de_hh_up_berufsfeuerwehr_EPSG_25832.json',
  'de_hh_up_berufsfeuerwehr_EPSG_4326.json',
  'de_hh_up_freiwillige_feuerwehr_EPSG_25832.json',
  'de_hh_up_freiwillige_feuerwehr_EPSG_4326.json',
  'settings.json'],
 'gew

In [28]:
# takes in a path to a file and returns the json object
def load_json(json_path, encoding='utf-8'):
    json_data = {}
    with open(json_path, 'r', encoding=encoding) as settings_file:
        json_data = json.load(settings_file)
    return json_data

In [29]:
m = folium.Map(location=(53.55, 9.99), zoom_start=12)

# Possible colors:

```{'lightgray', 'gray', 'beige', 'darkgreen', 'black', 'red', 'darkblue', 'darkred', 'lightblue', 'green', 'white', 'cadetblue', 'pink', 'lightred', 'lightgreen', 'darkpurple', 'purple', 'blue', 'orange'}```

In [30]:
for category, file_list in files.items():
    
    # Load the settings for this category if it exists.
    settings_path = os.path.join('data', category, 'settings.json')
    category_settings = load_json(settings_path) if os.path.exists(settings_path) else {}
    
    # Create a feature group for this category using the display_name from settings.
    feature_group_name = category_settings.get('display_name', category.replace('_json', '').replace('_', ' ').title())
    feature_group = folium.FeatureGroup(name=feature_group_name, show=False)

    marker_cluster = None
    
    # display the feature groups on the map
    for file_name in file_list:

        # Skip the settings.json file.
        if file_name == "settings.json":
            continue
        
        # Only add files with coordinates in EPSG:4326 for compatibility.
        if 'EPSG_4326' in file_name:
            geojson_path = os.path.join('data', category, file_name)
            geojson_data = load_json(geojson_path)
            # print("Loaded", geojson_path)
            
            file_settings = category_settings.get('files', {}).get(file_name, {})
            popup_properties = file_settings.get('popup_properties', {})
            
            for feature in geojson_data['features']:

                # compute the style for a geojson object
                def compute_style_function(file_settings):
                    """ Returns a style function configured with the given file settings. """
                    def compute_style(feature):
                        # Default color from settings
                        fillColor = file_settings.get('color', 'blue')
                        color = file_settings.get('color', 'black')
                        line_weight = file_settings.get('line_weight', 8)
                        
                        # If there's a colormap, compute the color based on feature value
                        if file_settings.get('colormap', False):
                            colormap_data = file_settings.get('colormap')
                            colormap_property = colormap_data["property"]
                            feature_value = feature['properties'][colormap_property]
                            colormap = cm.LinearColormap(colors=colormap_data["colors"], vmin=colormap_data["vmin"], vmax=colormap_data["vmax"])
                            color = colormap(feature_value)[:7]  # Remove alpha channel from color

                        return {
                            'fillColor': fillColor,
                            'color': color,
                            'weight': line_weight
                        }
                    return compute_style

                # populate the popup content with the properties specified in settings
                # The first line is the name of the marker
                marker_name = file_settings.get('name', '')
                popup_content = f'<b>{marker_name}</b><br>'

                # Afterwards, each line is a property of the marker, as specified in settings.json
                popup_content += '<br>'.join(
                    [f"<b>{display_name}</b>: {feature['properties'][internal_name]}" for display_name, internal_name in popup_properties.items() if internal_name in feature['properties']]
                )

                feature_type = file_settings.get('type')
                
                # Add the feature to the map. Feature type is specified in settings.json
                if feature_type == "geojson":

                    style_function = compute_style_function(file_settings)

                    folium.GeoJson(
                        feature,
                        style_function=style_function,
                        tooltip=popup_content
                    ).add_to(feature_group)
                
                elif feature_type == "marker":
                    coords = feature['geometry']['coordinates']

                    # get the icon data from settings.json
                    icon = file_settings.get('icon', ''), 
                    icon_prefix = file_settings.get('icon-prefix', '')
                    icon_color = file_settings.get('color', 'blue')

                    folium.Marker(
                        location=[coords[1], coords[0]],  # Swap coords for latitude and longitude
                        popup=popup_content,
                        icon=folium.Icon(icon=icon, prefix=icon_prefix, color=icon_color)
                    ).add_to(feature_group)
                
                elif feature_type == "marker-cluster":

                    if marker_cluster is None:
                        marker_cluster = MarkerCluster().add_to(feature_group)

                    coords = feature['geometry']['coordinates']

                    # get the icon data from settings.json
                    icon = file_settings.get('icon', ''), 
                    icon_prefix = file_settings.get('icon-prefix', '')
                    icon_color = file_settings.get('color', 'blue')

                    folium.Marker(
                        location=[coords[1], coords[0]],  # Swap coords for latitude and longitude
                        popup=popup_content,
                        icon=folium.Icon(icon=icon, prefix=icon_prefix, color=icon_color)
                    ).add_to(marker_cluster)
                
                elif feature_type == "choropleth":
                    # TODO: Add choropleth logic here.
                    pass

    feature_group.add_to(m)

folium.LayerControl().add_to(m)

In [31]:
# m

In [32]:
# save the map to an html file
m.save('map.html')